In [ ]:
# by HoRan
import os
import pathlib
from pathlib import Path
import re
import json
import pandas as pd

def get_content(path):
    dir_path = path
    if dir_path == "":
        dir_path = os.getcwd()

    dir_list = list()
    file_list = list()
    entries = Path(dir_path)

    entries = sorted(entries.iterdir(),key = lambda s: s.name)

    for entry in entries:
        if entry.is_file():
            file_list.append(entry)
        if entry.is_dir():
            dir_list.append(entry)

    return dir_list, file_list

def get_child_content(path):
    dir_list, file_list = get_content(path)

    new_dir_list = list() 
    new_file_list = list() 

    for i,dir in enumerate(dir_list, 0):
        name = path+"\\"+dir.name
        new_dir_list.append(name)

    for i, file in enumerate(file_list, 0):
        name = path+"\\"+file.name
        new_file_list.append(name)

    re_dir = new_dir_list
    re_file = new_file_list

    return re_dir, re_file  

dir_list, file_list = get_child_content('.')

In [ ]:

str_ = file_list[1]
print(str_)
st = "(.*?)\\\g\((.*?)\)_d\((.*?)\)_b\((.*?)\)_m\((.*?)\)_l\((.*?)\)_proj\((.*?)\)_d\((.*?)\)"
data = re.findall(st, str_)
print(data)


In [ ]:
result_info = list()
for i in file_list:
    if '.json' in i:
        data = re.findall(st,i)
    else:
        continue
    info = {
        "token": data[0][0],
        "gpu_num": int(data[0][1]),
        "dataset": data[0][2],
        "batch_size": data[0][3],
        "model": data[0][4],
        "layers": int(data[0][5]),
        "path": i
    }
    
    result_info.append(info)


In [ ]:
output_data = list()
for idx, _ in enumerate(result_info):
    with open(result_info[idx]['path']) as json_file:
        json_contect = json_file.read()
        model_result = json.loads(json_contect)
        model_hyper = model_result["model hpyerparameter"]
        model_stat = model_result["statistics"]

        model_info = {
            "model_name": model_hyper["model"],
            "dataset": model_hyper["dataset"],
            "proj_type": model_hyper["proj_type"],
            "batch_size": model_hyper["train_bsz"],
            "layers": model_hyper["layers"], 
            "max_len": model_hyper["max_len"],
            "batch_size": model_hyper["train_bsz"],
            "gpu_num": len(set((model_hyper["gpu_ids"]).split(','))),
            "multi_t": str(model_hyper["multi_t"]),
            "best test acc. mean": model_stat["best test acc. mean"],
            "best test acc. std": model_stat["best test acc. std"],
            "best epoch mean": model_stat["best epoch mean"],
            "best epoch std": model_stat["best epoch std"],
            "epoch_time mean": model_stat["epoch_time mean"],
            "epoch_time std": model_stat["epoch_time std"],
            "runtime mean": model_stat["runtime mean"],
            "runtime std": model_stat["runtime std"],
        }
        output_data.append(model_info)

In [ ]:
model_pd = pd.DataFrame(output_data)
output_path = "result.csv"
model_pd.to_csv(output_path, index=False)